In [1]:
import pandas as pd

In [ ]:
!wget http://www.lawrence.edu/fast/greggj/CMSC490/github.zip
!unzip github.zip

In [2]:
data_frame = pd.read_json('embold_train.json')

In [3]:
train_df= pd.read_json("embold_train.json").reset_index(drop=True)
train_df.head()

,title,body,label
0,y-zoom piano roll,a y-zoom on the piano roll would be useful.,1
1,buggy behavior in selection,! screenshot from 2016-02-23 21 27 40 https:/...,0
2,auto update feature,"hi,\r \r great job so far, @saenzramiro ! : \r...",1
3,filter out noisy endpoints in logs,i think we should stop logging requests to:\r ...,1
4,enable pid on / pid off alarm actions for ardu...,expected behavior\r alarm actions pid on and p...,0


In [4]:
test_df= pd.read_json("embold_test.json").reset_index(drop=True)
test_df.head()

,title,body
0,config question path-specific environment var...,issue description or question\r \r hey @artemg...
1,crash indien vol,de simulator crasht als hij vol zit
2,unable to mine rocks,"sarkasmo starting today, when i hit enter act..."
3,not all whitelists are processed,create following rules... order of creation is...
4,add ctx menu for idafree 70 and idafree 5,"associated with .dll, .dll_, .exe, .exe_, .sc,..."


In [5]:
train_ex_df= pd.read_json("embold_train_extra.json").reset_index(drop=True)
train_ex_df.head()

,title,body,label
0,use a 8bit typeface,since this is meant to emulate some old arcade...,1
1,implement wireless m-bus binding,_from chris.pa...@googlemail.com https://cod...,1
2,add multilang support for timeago.js,currently it is only en . \r required to add ...,1
3,scaleway - seg-fault on shutdown,tbr irc creates a new scaleway instance with...,0
4,sistema de pintura: no se guardar los nuevos p...,este sp ya estaba asignado a un carro y se enc...,0


In [6]:
def dataset_length_check(data_frame):
    print(len(data_frame),data_frame.index.shape[-1])
                 
dataset_length_check(train_df)
dataset_length_check(train_ex_df)
dataset_length_check(test_df)

150000 150000
300000 300000
30000 30000


In [7]:
data_frame = pd.concat([train_df,train_ex_df],ignore_index=True)
data_frame.head()

,title,body,label
0,y-zoom piano roll,a y-zoom on the piano roll would be useful.,1
1,buggy behavior in selection,! screenshot from 2016-02-23 21 27 40 https:/...,0
2,auto update feature,"hi,\r \r great job so far, @saenzramiro ! : \r...",1
3,filter out noisy endpoints in logs,i think we should stop logging requests to:\r ...,1
4,enable pid on / pid off alarm actions for ardu...,expected behavior\r alarm actions pid on and p...,0


In [8]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450000 entries, 0 to 449999
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   title   450000 non-null  object
 1   body    450000 non-null  object
 2   label   450000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 10.3+ MB


In [9]:
df_bug = data_frame[data_frame['label']==0]
df_feature = data_frame[data_frame['label']==1]
df_question = data_frame[data_frame['label']==2]

In [10]:
label_counts = data_frame.label.value_counts().sort_index()
label_counts

0    200481
1    207318
2     42201
Name: label, dtype: int64

In [11]:
def fx(x):
    return x['title'] + " " + x['body']   
data_frame['text']= data_frame.apply(lambda x : fx(x),axis=1)
data_frame['text']= data_frame.apply(lambda x : fx(x),axis=1)

In [12]:
print('Number of datapoints with label as Bug :',label_counts[0])
print('Number of datapoints with label as Feature :',label_counts[1])
print('Number of datapoints with label as Question :',label_counts[2])

Number of datapoints with label as Bug : 200481
Number of datapoints with label as Feature : 207318
Number of datapoints with label as Question : 42201


In [13]:
import nltk
import re
import string


def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)

    return text

In [14]:
from string import punctuation
from nltk.corpus import stopwords

def remove_stopwords(git_text):
    # filters charecter-by-charecter : ['h', 'e', 'e', 'l', 'o', 'o', ' ', 'm', 'y', ' ', 'n', 'a', 'm', 'e', ' ', 'i', 's', ' ', 'p', 'u', 'r', 'v', 'a']
    remove_punctuation = [ch for ch in git_text if ch not in punctuation]
    # convert them back to sentences and split into words
    remove_punctuation = "".join(remove_punctuation).split()
    filtered_git_text = [word.lower() for word in remove_punctuation if word.lower() not in stopwords.words('english')]
    return filtered_git_text

In [15]:
data_frame['text'] = data_frame['text'].apply(lambda x: x.replace("\\r", ""))
data_frame['text'] = data_frame['text'].apply(lambda x: clean_text(x))
data_frame.head()

,title,body,label,text
0,y-zoom piano roll,a y-zoom on the piano roll would be useful.,1,yzoom piano roll a yzoom on the piano roll wou...
1,buggy behavior in selection,! screenshot from 2016-02-23 21 27 40 https:/...,0,buggy behavior in selection screenshot from ...
2,auto update feature,"hi,\r \r great job so far, @saenzramiro ! : \r...",1,auto update feature hi great job so far saenz...
3,filter out noisy endpoints in logs,i think we should stop logging requests to:\r ...,1,filter out noisy endpoints in logs i think we ...
4,enable pid on / pid off alarm actions for ardu...,expected behavior\r alarm actions pid on and p...,0,enable pid on pid off alarm actions for expe...


In [16]:
data_frame.describe

<bound method NDFrame.describe of                                                     title  \
0                                       y-zoom piano roll   
1                             buggy behavior in selection   
2                                     auto update feature   
3                      filter out noisy endpoints in logs   
4       enable pid on / pid off alarm actions for ardu...   
...                                                   ...   
449995                       shield against invalid dates   
449996                                     fix the knight   
449997  portaudio playback binding crashes often, ubun...   
449998  add --db-path option for allowing user to spec...   
449999                 staff users cannot log in to admin   

                                                     body  label  \
0             a y-zoom on the piano roll would be useful.      1   
1       ! screenshot from 2016-02-23 21 27 40  https:/...      0   
2       hi,\r \r great job so

In [17]:
import pandas as pd
import numpy as np

In [18]:
texts = data_frame['text'].values
labels = data_frame['label'].values

In [19]:
texts = np.array(texts)
labels = np.array(labels)

In [20]:
X_train = texts[:270000]
x_test = texts[270000:360000]
x_val = texts[360000:]

Y_train = labels[:270000]
y_test = labels[270000:360000]
y_val = labels[360000:]


In [21]:
import tensorflow as tf

2023-10-18 14:32:05.306804: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-18 14:32:05.334609: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train,Y_train)).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(64)
val_ds = tf.data.Dataset.from_tensor_slices((x_val,y_val)).batch(64)

2023-10-18 14:32:07.717468: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-18 14:32:07.737426: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-18 14:32:07.737521: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [23]:
from tensorflow.keras import layers

In [24]:
text_only_train_ds = train_ds.map(lambda x, y: x)

max_length = 100
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)

In [25]:
from tensorflow import keras

In [27]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(3, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot_1 (TFOpLambda)   (None, None, 20000)       0         
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                5128448   
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 5128643 (19.56 MB)
Trainable params: 5128643 (19.56 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [28]:
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10)

Epoch 1/10


2023-10-18 14:32:35.488717: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8902
2023-10-18 14:32:35.516342: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-18 14:32:35.540758: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe1532771f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-18 14:32:35.540778: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4080, Compute Capability 8.9
2023-10-18 14:32:35.543297: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-18 14:32:35.594742: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

4219/4219 [==============================] - 171s 40ms/step - loss: 0.5844 - accuracy: 0.7751 - val_loss: 0.5267 - val_accuracy: 0.8009
Epoch 2/10
4219/4219 [==============================] - 172s 41ms/step - loss: 0.5248 - accuracy: 0.8058 - val_loss: 0.5155 - val_accuracy: 0.8057
Epoch 3/10
4219/4219 [==============================] - 173s 41ms/step - loss: 0.5087 - accuracy: 0.8127 - val_loss: 0.5117 - val_accuracy: 0.8059
Epoch 4/10
4219/4219 [==============================] - 163s 39ms/step - loss: 0.4971 - accuracy: 0.8168 - val_loss: 0.5112 - val_accuracy: 0.8069
Epoch 5/10
4219/4219 [==============================] - 110s 26ms/step - loss: 0.4868 - accuracy: 0.8213 - val_loss: 0.5206 - val_accuracy: 0.8044
Epoch 6/10
4219/4219 [==============================] - 114s 27ms/step - loss: 0.4777 - accuracy: 0.8249 - val_loss: 0.5155 - val_accuracy: 0.8062
Epoch 7/10
4219/4219 [==============================] - 110s 26ms/step - loss: 0.4681 - accuracy: 0.8291 - val_loss: 0.5189 - val